In [1]:
import numpy as np
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import optuna
%matplotlib inline

In [2]:
def print_weight_map(weight, features):
    # feature importance
    feature_imp = weight.reshape(features.shape)
    feature_imp_arg = np.argsort(feature_imp)
    f_num = len(feature_imp_arg)
    plt.rcParams["font.family"] = "IPAexGothic"
    plt.figure(figsize=(20,20))
    plt.barh(range(len(feature_imp_arg)), feature_imp[feature_imp_arg], color='b', align='center')
    plt.yticks(range(len(feature_imp_arg)), features[feature_imp_arg])
    plt.show()

In [3]:
def plot_roc_curve(fpr, tpr, auc):
    # ROC曲線をプロット
    plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
    plt.legend()
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.show()

In [11]:
# optunaで最適なC_valを探す

def train(trial):
    
    
    path = '../../data/dataframes/std_data/'
    
    th_prob = 0.5
    C_val = trial.suggest_uniform('C_val', 1, 100)
#     l1_ratio = trial.suggest_uniform('l1_ratio', 0, 1)
    
    print(" C_val:{}".format(C_val))
    
    cm_all = np.zeros((2, 2))
    
    # 予測した確率全体を格納
    probs_all = np.array([])
    y_true_all = np.array([])
    
    for y in range(1978, 2020):
        
        # データの生成
        x_train_std = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).values
        x_test_std = pd.read_pickle(path +'test/{}_x.pkl'.format(str(y))).values
        y_train = pd.read_pickle(path +'train/{}_y.pkl'.format(str(y))).values
        y_test = pd.read_pickle(path +'test/{}_y.pkl'.format(str(y))).values
        
        features = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).columns
        
        # 学習
        lr = LogisticRegression(class_weight="balanced", solver="liblinear",  penalty="l2", C=C_val) # ロジスティック回帰モデルのインスタンスを作成
        lr.fit(x_train_std, y_train) # ロジスティック回帰モデルの重みを学習
        
        # 予測(確率)
        probs = lr.predict_proba(x_test_std)
        probs_all = np.hstack((probs_all, probs[:,1]))
        y_true_all = np.hstack((y_true_all, y_test))

        # 混同行列
        y_pred = np.where((probs[:,1] ==  max(probs[:,1])) | (probs[:,1] > th_prob), 1, 0) #確率→0/1
        cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
        cm_all += cm
        
       # 各特徴量の重み係数
        weight = lr.coef_
        
        
#         print_weight_map(weight, features)

    auc = roc_auc_score(y_true_all, probs_all)
    fpr, tpr, thresholds = roc_curve(y_true_all, probs_all)
    
#     print_weight_map(weight, features)
#     plot_roc_curve(fpr, tpr, auc)
    
#     print("len: {0} , {1}".format(len(y_true_all), len(probs_all) ))
#     print("confusion_matrix: ")
#     print(cm_all)
#     print("AUC: ")
#     print(auc)
#     print()
    return -auc

In [12]:
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
@ignore_warnings(category=ConvergenceWarning)
def main():
    np.set_printoptions(precision=3, suppress=True)
    study = optuna.create_study()
    study.optimize(train, n_trials=100)
    print(study.best_trial)

In [13]:
if __name__ == "__main__":
    main()

 C_val:43.92664348159629


[I 2019-08-22 09:58:14,145] Finished trial#0 resulted in value: -0.6873062834601297. Current best value is -0.6873062834601297 with parameters: {'C_val': 43.92664348159629}.


 C_val:22.547390753895062


[I 2019-08-22 09:58:14,576] Finished trial#1 resulted in value: -0.6926599041983657. Current best value is -0.6926599041983657 with parameters: {'C_val': 22.547390753895062}.


 C_val:64.60569137197811


...

 C_val:3.5936957122220647


[I 2019-08-22 09:59:02,493] Finished trial#98 resulted in value: -0.66701887855734. Current best value is -0.694632290786137 with parameters: {'C_val': 12.346329899775844}.


 C_val:22.35030262696173


[I 2019-08-22 09:59:02,960] Finished trial#99 resulted in value: -0.6929416737109044. Current best value is -0.694632290786137 with parameters: {'C_val': 12.346329899775844}.


FrozenTrial(number=27, state=<TrialState.COMPLETE: 1>, value=-0.694632290786137, datetime_start=datetime.datetime(2019, 8, 22, 9, 58, 26, 290036), datetime_complete=datetime.datetime(2019, 8, 22, 9, 58, 26, 761405), params={'C_val': 12.346329899775844}, distributions={'C_val': UniformDistribution(low=1, high=100)}, user_attrs={}, system_attrs={'_number': 27}, intermediate_values={}, params_in_internal_repr={'C_val': 12.346329899775844}, trial_id=27)


# 結果
C = 0.001

In [74]:
# best value is -0.7222457030149338 with parameters: {'C_val': 0.0013591999321332033}.
# best value is -0.7420400112707805 with parameters: {'C_val': 0.08453576485123458, 'l1_ratio': 0.3231149632136399}.